In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
import datetime
import time

In [3]:
#url = 'https://apihub.kma.go.kr/api/typ01/url/awsh.php?var=RN&tm=202304101500&stn=548&help=0&authKey=aaZrhx_QRmyma4cf0PZsug'
url = 'https://apihub.kma.go.kr/api/typ01/url/awsh.php?var=RN&tm=202304101500&stn=548&help=1&authKey=aaZrhx_QRmyma4cf0PZsug'
res = requests.get(url)
res.raise_for_status()

In [4]:
time.sleep(2)

In [5]:
soup = BeautifulSoup(res.text, 'html')

In [6]:
print(soup)
print(type(soup))
print(type(soup.text))

<html><body><p>#START7777
#--------------------------------------------------------------------------------------------------
#  RE_SUM        : 과거 60분간 '강수있음'의 분수 합 
#  RE_QCM        : 과거 60분간 사용된 '강수유무' 자료수 합 
#  RN_DAY        : 해당시간까지의 일강수량 (mm) 
#  RN_DAY_MI     : 위 일강수량의 시간차 : 관측시각-TM (분) 
#  RN_HR1        : 1시간 강수량 (mm) 
#  RN_HR1_MI     : 관측시간-1시간 강수량 누적 시작 시간차 (분) 
#  RN_60M_MAX    : 과거 60분간에 최대 60분이동누적강수량 (mm) 
#  RN_60M_MAX_MI : 위 최대값의 시간차 : 관측시각-TM (분) 
#  RN_60M_QCM    : 위 계산에 사용된 60분 강수량 자료수 
#  RN_15M_MAX    : 과거 60분간에 최대 15분이동누적강수량 (mm) 
#  RN_15M_MAX_MI : 위 최대값의 시간차 : 관측시각-TM (분) 
#  RN_15M_QCM    : 위 계산에 사용된 15분 강수량 자료수 
#--------------------------------------------------------------------------------------------------
# YYMMDDHHMI   STN  RE  RE     RN  MI     RN  MI     RN  MI QCM     RN  MI QCM
#                  SUM QCM    DAY DAY    HR1 HR1    60M 60M 60M    15M 15M 15M
#        KST    ID  mi  mi     mm  mi     mm  mi     mm  mi  mi     mm  mi  mi
202304101500   54

In [7]:
front = soup.text.split('\n')

In [8]:
type(front)

list

In [9]:
front[1]

'#--------------------------------------------------------------------------------------------------'

In [10]:
real  = []
for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
    if(front[i][0] != "#"):
        real.append(front[i])
        print(front[i-3])
        print(front[i-2])
        print(front[i-1]) #        KST    ID  mi  mi     mm  mi     mm  mi     mm  mi  mi     mm  mi  mi
        print(front[i])

# YYMMDDHHMI   STN  RE  RE     RN  MI     RN  MI     RN  MI QCM     RN  MI QCM
#                  SUM QCM    DAY DAY    HR1 HR1    60M 60M 60M    15M 15M 15M
#        KST    ID  mi  mi     mm  mi     mm  mi     mm  mi  mi     mm  mi  mi
202304101500   548   0  60    0.0   0    0.0 -60    0.0 -59  60    0.0 -59  60


In [11]:
real

['202304101500   548   0  60    0.0   0    0.0 -60    0.0 -59  60    0.0 -59  60']

##변수 조립##

# 기상청 API 설명
https://apihub.kma.go.kr/



# AWS2 매분자료
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?tm2=202302010900&stn=0&disp=0&help=1&authKey=LcdJI2t7SuCHSSNrexrgGA

MD_column = [ 'YYMMDDHHMI_KST', 'STN_ID', 'WD1_deg', 'WS1_m/s', 'WDS_deg', 'WSS_m/s', 'WS10_deg', 'WS10_m/s', 'TA_C', 
'RE_1', 'RN-15m_mm', 'RN-60m_mm', 'RN-12H_mm', 'RN-DAY_mm', 'HM_%', 'PA_hPa', 'PS_hPa', 'TD_C' ]


# AWS2 매분 시정 자료(현천도 나옴)
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?tm2=202304141510&stn=548&disp=0&help=1&authKey=LcdJI2t7SuCHSSNrexrgGA

MD_NW_column = [ 'YYMMDDHHMI_KST',  'STN_ID', 'LON_deg', 'LAT_deg  S', 'VIS1_m', 'VIS10_m', 'WW1', 'WW15' ]


# AWS2 매분 현천 자료(추가 처리 필요)
https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?

https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?tm1=202304141200&tm2=202304141240&itv=10&range=240&stn=548&help=1&authKey=LcdJI2t7SuCHSSNrexrgGA

In [12]:
url = 'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_ww1?tm2=201503221200&itv=60&range=60&stn=548&help=1&authKey=aaZrhx_QRmyma4cf0PZsug'
res = requests.get(url)
res.raise_for_status()

time.sleep(2)

soup = BeautifulSoup(res.text, 'html')

In [13]:
print(soup)
print(type(soup))
print(type(soup.text))

<html><body><p>#START7777
#--------------------------------------------------------------------------------------------------
#  S   : Àåºñ±¸ºÐ (1:¾È°³°üÃø¸Á(¸ÕÀú ¼³Ä¡), 2:Ã·´ÜÈ­Àåºñ)
#  N   : range ±â°£ µ¿¾È¿¡ °üÃøµÈ Çö»óÁ¾·ùÀÇ ¼ö
#  WW1 : Çö»ó¹øÈ£
#  NN1 : WW1 Çö»óÀÇ °üÃøµÈ È½¼ö(ºÐ)
#  ... : WW1, NN1 ÀÌ N °¹¼ö¸¸Å­ ¹Ýº¹µÊ
#--------------------------------------------------------------------------------------------------
# ÇöÃµÄÚµå : 0~2(¸¼À½), 4(¿¬¹«), 10(¹Ú¹«), 30(¾È°³), 40~42(ºñ), 50~59(¾È°³ºñ), 60~68(ºñ), 71~76(´«)
#--------------------------------------------------------------------------------------------------
# YYMMDDHHMI,  STN,         LON,         LAT, S, N, WW1, NN1, ...,=
#        KST,   ID,       (deg),       (deg),  ,  ,    ,    , ...,=
201503221200,  548,127.63954926, 37.26882935, 2, 2,   0,  57,  10,   3,=
#7777END
</p></body></html>
<class 'bs4.BeautifulSoup'>
<class 'str'>


In [14]:
front = soup.text.split('\n')

In [15]:
type(front)

list

In [16]:
front[1]

'#--------------------------------------------------------------------------------------------------'

In [17]:
real  = []
for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
    if(front[i][0] != "#"):
        real.append(front[i])
        print(front[i-1]) #        KST    ID  mi  mi     mm  mi     mm  mi     mm  mi  mi     mm  mi  mi
        print(front[i])

#        KST,   ID,       (deg),       (deg),  ,  ,    ,    , ...,=
201503221200,  548,127.63954926, 37.26882935, 2, 2,   0,  57,  10,   3,=


In [18]:
if(len(real) > 0):
    real[0]

In [19]:
real

['201503221200,  548,127.63954926, 37.26882935, 2, 2,   0,  57,  10,   3,=']

In [20]:
import requests


##CCTV 기반 날씨
url = 'http://apis.data.go.kr/1360000/RoadWthrInfoService/getCctvStnRoadWthr'
params ={'serviceKey' : 'EEa94+B5XdFwdYp09nxCOC1JmGOLfawvDQTOrJMqfJsRihjPY5K75Oap+IWUktv/Zq89XHAsX2Y+c80TAkdovA==', 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'XML', 'eqmtId' : '0500C00001','hhCode' : '00' }

response = requests.get(url, params=params)
print(response.content)
response.text

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><baseDate>20230422</baseDate><baseTime>1705</baseTime><weatherNm>\xed\x98\x84\xec\x83\x81\xec\x97\x86\xec\x9d\x8c</weatherNm></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>1</totalCount></body></response>\r\n'


'<?xml version="1.0" encoding="UTF-8"?>\r\n<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><baseDate>20230422</baseDate><baseTime>1705</baseTime><weatherNm>현상없음</weatherNm></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>1</totalCount></body></response>\r\n'

In [21]:
'''
stn = '548'
api_key = 'LcdJI2t7SuCHSSNrexrgGA'

u = 'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?'
stn = 'stn=' + stn
api_key = 'authKey=' + api_key

url = u + '&' + stn  + '&' + api_key
res = requests.get(url)
res.raise_for_status()
time.sleep(1)

soup = BeautifulSoup(res.text, 'html')
front = soup.text

front
'''

"\nstn = '548'\napi_key = 'LcdJI2t7SuCHSSNrexrgGA'\n\nu = 'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?'\nstn = 'stn=' + stn\napi_key = 'authKey=' + api_key\n\nurl = u + '&' + stn  + '&' + api_key\nres = requests.get(url)\nres.raise_for_status()\ntime.sleep(1)\n\nsoup = BeautifulSoup(res.text, 'html')\nfront = soup.text\n\nfront\n"

In [22]:
def predict_part_load(local, api_key):
    #예보 구역 이름과 api 키를 입력받아서
    #현재 지역의 예보 구역 코드를 리턴. 
    '''
    REG_ID   : 예보구역코드
    TM_ST    : 시작시각(년월일시분,KST)
    TM_ED    : 종료시각(년월일시분,KST)
    REG_SP   : 특성 (A:육상광역,B:육상국지,C:도시,D:산악,E:고속도로,H:해상광역,I:해상국지,J:연안바다,K:해수욕장,L:연안항로,M:먼항로,P:산악)
    REG_NAME : 예보구역명


    11B20701 201610131800 210012310000 C      이천 
    11B20702 201610131800 210012310000 C      광주 
    11B20703 201610131800 210012310000 C      여주 
    11C000D1 199001010000 210012310000 D      계룡산
    '''

    url = f'https://apihub.kma.go.kr/api/typ01/url/fct_shrt_reg.php?tmfc=0&authKey={api_key}'
    
    res = requests.get(url)
    #time.sleep(1)

    soup = BeautifulSoup(res.text, 'html')
    front = soup.text.split('\n')

    real  = []
    for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
        if(front[i][0] != "#"):
            real.append(front[i])

    location = None

    for i in range(len(real)):
        real[i] = str(real[i]).split(' ') # 슬라이싱
        for k in range(len(real[i])):
            if real[i][k] == local:
                location = real[i]

    if location == None: 
        print('해당 지역 정보가 없음.')
        return None

    info = []
    for i in range(len(location)):
        if len(location[i]) > 0:
            info.append(location[i])

    return info

In [23]:
p = predict_part_load('여주', 'LcdJI2t7SuCHSSNrexrgGA')
print(p)
print(type(p))

['11B20703', '201610131800', '210012310000', 'C', '여주']
<class 'list'>


In [24]:
p = predict_part_load('ㅇㅇ', 'LcdJI2t7SuCHSSNrexrgGA')
print(p)
print(type(p))

해당 지역 정보가 없음.
None
<class 'NoneType'>


In [25]:
#https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl2.php?reg=11B20703&tmfc1=202304200000&tmfc2=202304210000&&disp=0&help=1&authKey=Os5DnJkMQpmOQ5yZDLKZ2Q

##기상청 날씨누리 단기 에보.

https://www.weather.go.kr/w/weather/forecast/short-term.do#




##기상청 초단기예보 API(기상청 API 허브에 나온 API랑 다름.)

https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [26]:
#http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst


api_key2 = 'eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D' #설정파일로 설정할 값
today = '20230421' #현재 시간으로 설정할 값
today_time = '0537' # 현재 시간으로 설정할 값
location_x = '55'  #설정파일로 설정할 값
location_y = '127' # 설정파일로 설정할 값
h = 2 # 몇시간 앞 정보까지 가져올 것인지(1이면 1시간 후, 2 이면 2시간 후)
load_h = h * 12

url = f'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={api_key2}&numOfRows={load_h}&pageNo=1&base_date={today}&base_time={today_time}&nx={location_x}&ny={location_y}'  
print(url)
res = requests.get(url)
soup = BeautifulSoup(res.text, 'xml')
#front = soup.text.split('\n')
data = soup.select('item')

#print(soup)
for it in data:
    print(it.text)

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=eqTbu5tgsBlxHlliXVpp4A6FZyGJCS6ct%2FQWVPZuhRy3x05IGhAHlr0U9b%2B4qPEUNfrWVf2n5FtdffkCi33MQg%3D%3D&numOfRows=24&pageNo=1&base_date=20230421&base_time=0537&nx=55&ny=127
202304210500TMP2023042106001055127
202304210500UUU2023042106000.655127
202304210500VVV202304210600-3.955127
202304210500VEC20230421060035155127
202304210500WSD2023042106003.955127
202304210500SKY202304210600155127
202304210500PTY202304210600055127
202304210500POP202304210600055127
202304210500WAV202304210600055127
202304210500PCP202304210600강수없음55127
202304210500REH2023042106005055127
202304210500SNO202304210600적설없음55127
202304210500TMP2023042107001155127
202304210500UUU2023042107000.255127
202304210500VVV202304210700-4.255127
202304210500VEC20230421070035755127
202304210500WSD2023042107004.255127
202304210500SKY202304210700155127
202304210500PTY202304210700055127
202304210500POP202304210700055127
202304210500WAV202304210700055127
20230421050

In [7]:
# 적설, 기간설정해서 조회, 여주는 없음.
# https://apihub.kma.go.kr/api/typ01/url/kma_snow2.php?tm=202212250000&tm_st=20221220000&stn=93&help=1&authKey=aF29N6-AQoKdvTevgNKChw #제공 o
# https://apihub.kma.go.kr/api/typ01/url/kma_snow2.php?tm=202212250000&tm_st=20221220000&stn=548&help=1&authKey=aF29N6-AQoKdvTevgNKChw #제공 x

# 적설 관측지점 조회
# https://apihub.kma.go.kr/api/typ01/url/stn_snow.php?mode=0&help=1&authKey=aF29N6-AQoKdvTevgNKChw

# 적설 조회, 특정 지점만 조회 불가능. 
# https://apihub.kma.go.kr/api/typ01/url/kma_snow1.php?sd=tot&tm=202212250000&help=1&authKey=aF29N6-AQoKdvTevgNKChw


# 매 분 자료 조회(구간 지정)
# https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?tm1=202301010000&tm2=202301010100&stn=548&disp=0&help=1&authKey=FouhOAZzSeeLoTgGcynndA
# https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?tm1=202301010000&tm2=202301010100&stn=548&disp=1&help=0&authKey=FouhOAZzSeeLoTgGcynndA



# 종합지상관측(매우 많은 정보 제공, 정시 자료만 제공, 다만 여주는 대상 아님.)
# https://apihub.kma.go.kr/api/typ01/url/kma_sfctm3.php?tm1=201512110100&tm2=201512140000&stn=108&help=1&authKey=MiEgBj40QL6hIAY-NLC-BQ

str